# Visual BERT

In [ ]:
import os
from dotenv import load_dotenv

# Загрузка переменных из файла .env
load_dotenv()

def get_hf_token():
    """
    Читает токен Hugging Face из переменной окружения HF_TOKEN.
    Возвращает токен или вызывает исключение, если токен отсутствует.
    """
    hf_token = os.getenv("HF_TOKEN")
    if not hf_token:
        raise ValueError("Токен Hugging Face (HF_TOKEN) не найден в переменных окружения. "
                         "Добавьте его в файл .env или передайте явно.")
    return hf_token

token = get_hf_token()
print("Hugging Face токен успешно загружен:", token)



In [2]:
from pathlib import Path

import torch
from transformers import pipeline
import matplotlib.pyplot as plt
from PIL import Image

from models.base_model import TextGenerationModelLoader, ImageToTextModelLoader, VQAModelLoader


class ImageCaptioningWithTranslation(ImageToTextModelLoader):
    """
    Класс для описания изображений и перевода описания на русский язык.
    Использует модель генерации описаний изображений и модель перевода текста.
    """
    def __init__(self, image_caption_model="Salesforce/blip-image-captioning-base",
                 translation_model="Helsinki-NLP/opus-mt-en-ru",
                 cache_dir="./data/cache", use_gpu=True, hf_token=None):
        # Инициализация модели для описания изображений
        super().__init__(model_name=image_caption_model, cache_dir=cache_dir, use_gpu=use_gpu, hf_token=hf_token)

        # Инициализация модели перевода текста
        self.translation_pipeline = pipeline(
            "translation_en_to_ru",
            model=translation_model,
            device=0 if use_gpu and torch.cuda.is_available() else -1,
            use_auth_token=hf_token
        )
        print(f"Модель перевода {translation_model} успешно загружена.")

    def describe_and_translate(self, image_path):
        """
        Генерирует описание изображения и переводит его на русский язык.
        
        :param image_path: Путь к изображению.
        :return: Описание изображения на русском языке.
        """
        if not Path(image_path).exists():
            raise FileNotFoundError(f"Изображение не найдено: {image_path}")

        # Генерация описания изображения
        description = self.generate_image_description(image_path)
        print(f"Сгенерированное описание на английском: {description}")

        # Перевод описания на русский язык
        translation = self.translation_pipeline(description)[0]["translation_text"]
        return description, translation

    def display_image_with_caption(self, image_path, translation):
        """
        Отображает изображение с русским описанием.
        
        :param image_path: Путь к изображению.
        :param translation: Текст описания на русском языке.
        """
        if not Path(image_path).exists():
            raise FileNotFoundError(f"Изображение не найдено: {image_path}")

        image = Image.open(image_path)

        # Отображение изображения и перевода
        plt.figure(figsize=(8, 8))
        plt.imshow(image)
        plt.axis("off")
        plt.title(translation, fontsize=14, wrap=True)
        plt.show()

In [ ]:
captioning_model = ImageCaptioningWithTranslation(
    image_caption_model="Salesforce/blip-image-captioning-base",
    translation_model="Helsinki-NLP/opus-mt-en-ru",
)

image_path = "data/imgs/road.jpg"

# Генерация описания и перевода
description, translated_caption = captioning_model.describe_and_translate(image_path)

# Вывод текста перевода
print("Сгенерированное описание на русском языке:", translated_caption)

# Отображение изображения с описанием
captioning_model.display_image_with_caption(image_path, translated_caption)